# ResNet models for identifying anatomy and US orientation


This notebook provides a comprehensive explanation of the Python code designed for image classification using PyTorch. The code uses a ResNet-18 model to classify ultrasound images based on two combined labels: 'US probe orientation' and 'Anatomical location'.


# Code Explanation


### Libraries and Modules Used

1. `pandas`: For data manipulation and analysis. 
2. `torch` and `torchvision`: For deep learning functionalities.
3. `PIL (Pillow)`: For image processing.
4. `time`: For timing the code execution.
5. `sklearn.preprocessing`: For label encoding.

### Hyperparameters

- `EPOCHS`: Number of epochs for training.
- `BATCH_SIZE`: Size of the mini-batch.

### Device Configuration

The code checks the availability of Apple's Metal Performance Shaders (MPS) and sets the device accordingly.

### Data Preprocessing

1. **Reading the DataFrame**: Reads an Excel file containing metadata.
2. **Creating a Concatenated Label**: Combines 'US probe orientation' and 'Anatomical location' into a new label.
3. **Label Encoding**: Encodes the concatenated label into integers.

### Custom Dataset Class

Implements a custom `Dataset` class that reads the images and corresponding labels.

### Data Transformations

Applies resizing and tensor conversion to the images.

### Model, Loss, and Optimizer

- **Model**: ResNet-18 pretrained on ImageNet.
- **Loss**: Cross-Entropy Loss.
- **Optimizer**: Adam.

### Training Loop

Iterates through epochs and mini-batches to update the model parameters.

### Testing

Evaluates the model on the test dataset and calculates the accuracy.

### Elapsed Time

Calculates and prints the time taken for training and testing.

### Changing the Model and Weights

To change the model and weights to ResNet-50 or ResNet-152, replace:

```python
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to(device)
```

with either:

- For ResNet-50:

```python
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1).to(device)
```

- For ResNet-152:

```python
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1).to(device)
```


```python
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import time
from sklearn.preprocessing import LabelEncoder

# Hyperparameters
EPOCHS = 5
BATCH_SIZE = 16

# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
    print("CPU being used.")
    device = torch.device("cpu")
else:
    print("Apple MPS being used.")
    device = torch.device("mps:0")

# Initialize label encoder for the concatenated label
concat_encoder = LabelEncoder()

# Start the timer
start_time = time.time()

# Read the Excel file into a DataFrame
df = pd.read_excel('US_hip_dataset.xlsx')

# Create a concatenated label
df['Concat_Label'] = df['US probe orientation'] + '_' + df['Anatomical location']

# Fit the encoder on the new concatenated label
concat_encoder.fit(df['Concat_Label'])

# Filter based on Sample ID for training and testing
train_df = df[df['Sample ID'] == 10001]#.sample(100)
test_df = df[df['Sample ID'] == 10002]#.sample(100)


# Custom UltrasoundDataset class
class UltrasoundDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['File location']
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Encode concatenated label to integer
        concat_label = concat_encoder.transform([self.dataframe.iloc[idx]['Concat_Label']])[0]

        return image, concat_label  # Return concatenated label


# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Initialize datasets and dataloaders
train_dataset = UltrasoundDataset(dataframe=train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = UltrasoundDataset(dataframe=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to(device)

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(EPOCHS):
    print("Currently in epoch " + str(epoch + 1))
    for image, concat_label in train_loader:
        image = image.to(device)
        concat_label = concat_label.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, concat_label)
        loss.backward()
        optimizer.step()
    current_time = time.time()
    print(f"Elapsed time since starting: {current_time - start_time:.2f} seconds")

# Testing the model
model.eval()
correct = 0
total = 0
model.to(torch.device("cpu"))
with torch.no_grad():
    for image, label in test_loader:
        # image = image.to(device)
        # label = label.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

print(f'Accuracy of the model on the test images: {int(100 * correct / total)}%')

# Stop the timer
end_time = time.time()

# Print the elapsed time
print("Finished Training")
print(f"Elapsed time: {end_time - start_time:.2f} seconds")

```

# Conclusion


This notebook has elaborated on the Python code for image classification using a neural network model in PyTorch. The code efficiently performs the classification task with the ability to adapt to different ResNet architectures easily.
